<a href="https://colab.research.google.com/github/SaiRam-43/Python_list/blob/main/FN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
data=pd.read_csv("fake_news.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
data.shape

(20800, 5)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
data.isna().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [7]:
data = data.drop(['id'],axis=1)

In [8]:
data=data.fillna('')

In [9]:
data['content'] = data['title'] + '' + data['text'] # Combine 'title' and 'text' columns into 'content'
data = data.drop(['title', 'text'], axis=1)          # Drop 'title' and 'text' columns

In [10]:
data.head()

,author,label,content
0,Darrell Lucus,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,Daniel J. Flynn,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Consortiumnews.com,1,Why the Truth Might Get You FiredWhy the Truth...
3,Jessica Purkiss,1,15 Civilians Killed In Single US Airstrike Hav...
4,Howard Portnoy,1,Iranian woman jailed for fictional unpublished...


In [11]:
# Convert to lowercase
data['content'] = data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [12]:
# Remove punctuation

data['content'] = data['content'].str.replace('[^\w\s]','')

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
!pip install textblob

In [15]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [18]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [19]:
# Do lemmatization
from nltk.stem import WordNetLemmatizer
from textblob import Word
data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'].head()

,content
0,house dem aide: didn’t even see comey’s letter...
1,"flynn: hillary clinton, big woman campus - bre..."
2,truth might get firedwhy truth might get fired...
3,15 civilian killed single u airstrike identifi...
4,iranian woman jailed fictional unpublished sto...


In [20]:
X=data['content']
y=data['label']

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=45,stratify=y)

In [25]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(14560,)
(6240,)
(14560,)
(6240,)


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)

tfidf_vect.fit(data['content'])

xtrain_tfidf = tfidf_vect.transform(X_train)
xtest_tfidf = tfidf_vect.transform(X_test)